In [ ]:
# Full Interception Effectiveness Notebook (Modelo Estático + Defesa Dinâmica)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import kagglehub

# --- Parte 1: Carregamento de Dados ---
# Download latest version
path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

print("Path to dataset files:", path)

# Load data
df = pd.read_csv(f"{path}/missile_attacks_daily.csv")


df = df.dropna(subset=['launched', 'destroyed'])
df['K_obs'] = df['destroyed'] / df['launched']
df['year'] = pd.to_datetime(df['time_start'], format='mixed').dt.year


Path to dataset files: /home/enzo/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136


In [6]:

# --- Parte 2: Modelo Estático de Wilkening ---
def calculate_K(P_track, k, n):
    return P_track * (1 - (1 - k)**n)

def estimate_sspk(K_obs, P_track, n):
    with np.errstate(invalid='ignore'):
        return 1 - np.power(1 - K_obs / P_track, 1/n)

P_track = 0.95
n_interceptors = 3

df['estimated_sspk_static'] = estimate_sspk(df['K_obs'], P_track, n_interceptors)


In [ ]:

# --- Parte 3: Visualização da Estimativa Estática ---
plt.figure(figsize=(8,5))
sns.histplot(df['estimated_sspk_static'].dropna(), kde=True, bins=20)
plt.title("Distribuição de SSPK Estimado (Modelo Estático)")
plt.xlabel("SSPK Estimado")
plt.ylabel("Frequência")
plt.grid(True)
plt.tight_layout()
plt.show()


In [4]:

# --- Parte 4: Parâmetros médios por ano ---
yearly_params = {
    2022: {"P_track": 0.85, "n_interceptors": 2},
    2023: {"P_track": 0.90, "n_interceptors": 2},
    2024: {"P_track": 0.93, "n_interceptors": 2},
    2025: {"P_track": 0.95, "n_interceptors": 3},
}

def get_year_based_defense_params(year):
    if year in yearly_params:
        return yearly_params[year]["P_track"], yearly_params[year]["n_interceptors"]
    else:
        return 0.85, 2  # fallback


In [ ]:

# --- Parte 5: Modelo Dinâmico por Ano ---
P_list, n_list = [], []
for y in df['year']:
    P, n = get_year_based_defense_params(y)
    P_list.append(P)
    n_list.append(n)
df['P_track_dynamic'] = P_list
df['n_dynamic'] = n_list
df['estimated_sspk_dynamic'] = estimate_sspk(df['K_obs'], df['P_track_dynamic'], df['n_dynamic'])


In [ ]:

# --- Parte 6: Modelo de Machine Learning ---
ml_df = df.dropna(subset=['model', 'launched', 'destroyed'])
ml_df['model'] = ml_df['model'].astype('category').cat.codes
if 'carrier' in ml_df.columns:
    ml_df['carrier'] = ml_df['carrier'].astype('category').cat.codes
else:
    ml_df['carrier'] = 0
features = ['model', 'launched', 'carrier']
X = ml_df[features]
y = ml_df['K_obs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE do modelo de ML: {mse:.4f}")


In [ ]:

# --- Parte 7: Previsão de Ataques Iminentes ---
def predict_from_new_attacks(csv_path):
    new_df = pd.read_csv(csv_path)
    new_df['year'] = pd.to_datetime(new_df['time_start']).dt.year
    P_list, n_list = [], []
    for y in new_df['year']:
        P, n = get_year_based_defense_params(y)
        P_list.append(P)
        n_list.append(n)
    new_df['P_track'] = P_list
    new_df['n_interceptors'] = n_list
    sspk_assumed = 0.7
    new_df['K_predicted'] = new_df.apply(
        lambda row: calculate_K(row['P_track'], sspk_assumed, row['n_interceptors']),
        axis=1
    )
    return new_df[['time_start', 'model', 'launched', 'target', 'P_track', 'n_interceptors', 'K_predicted']]

# Exemplo:
# predicted_df = predict_from_new_attacks("/mnt/data/ataque_iminente.csv")
# print(predicted_df)
